# **ETL PROCESS**

## Objectives

* Fetch data and save it as raw data file and upload it to the workspace. Take the data through the ETL process to clean it and also include a scaled and encoded file for machine learning purposes.

## Inputs

* Raw CSV data file.

## Outputs

* This notebook will hope to generate a clean CSV file of the data as well as a scaled and encoded txt file. 

## Additional Comments

* This dataset was sourced from Kaggle and contains data regarding air pollution quality in the US in from 2000 to 2016.



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

# Section 1

## Extract and read the data.

In [1]:
# Import all necessary packages
import numpy as np
import pandas as pd
import sklearn as sk
import feature_engine as fe 
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

print("All packages imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sk.__version__}")
print(f"Feature-engine version: {fe.__version__}")

All packages imported successfully!
NumPy version: 1.26.1
Pandas version: 2.1.1
Scikit-learn version: 1.3.1
Feature-engine version: 1.6.1


In [ ]:
# Read data and return full DataFrame with shape to make sure everything is working
# Also return the data shape
df = pd.read_csv("../inputs/pollution_us_2000_2016.csv")
print("Data loaded successfully!")
print(f"DataFrame shape: {df.shape}")
df

Data loaded successfully!
DataFrame shape: (1746661, 29)


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.200,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.200,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.200,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.200,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.600,23,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1746656,24599,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-30,Parts per billion,...,Parts per billion,0.000000,0.0,2,NaN,Parts per million,0.091667,0.100,2,1.0
1746657,24600,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.067714,0.127,0,NaN
1746658,24601,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,-0.022727,0.0,0,0.0,Parts per million,0.100000,0.100,0,1.0
1746659,24602,56,21,100,NCore - North Cheyenne Soccer Complex,Wyoming,Laramie,Not in a city,2016-03-31,Parts per billion,...,Parts per billion,0.000000,0.0,5,NaN,Parts per million,0.067714,0.127,0,NaN


In [7]:
# Return the first five values of the DataFrame for future observation purposes where necessary
df.head()

,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
0,0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,1.145833,4.2,21,NaN
1,1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,3.000000,9.0,21,13.0,Parts per million,0.878947,2.2,23,25.0
2,2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,1.145833,4.2,21,NaN
3,3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-01,Parts per billion,...,Parts per billion,2.975000,6.6,23,NaN,Parts per million,0.878947,2.2,23,25.0
4,4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,Phoenix,2000-01-02,Parts per billion,...,Parts per billion,1.958333,3.0,22,4.0,Parts per million,0.850000,1.6,23,NaN


In [26]:
# Extract a random, fractioned sample of the data of 2500 values for analytic purposes
# Also return the new data shape
df1 = df.sample(frac=0.0014311, random_state=10)
print(f"DataFrame shape: {df1.shape}")
df1.head()

DataFrame shape: (2500, 29)


,Unnamed: 0,State Code,County Code,Site Num,Address,State,County,City,Date Local,NO2 Units,...,SO2 Units,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
69905,69905,42,3,10,CARNEGIE SCIENCE CENTER - 1 ALLEGHENY RD,Pennsylvania,Allegheny,Pittsburgh,2000-06-15,Parts per billion,...,Parts per billion,9.037500,16.3,11,NaN,Parts per million,0.000000,0.0,0,0.0
1573464,109052,42,49,3,10TH AND MARNE STREETS,Pennsylvania,Erie,Erie,2014-10-13,Parts per billion,...,Parts per billion,1.041667,5.0,23,7.0,Parts per million,0.054167,0.1,11,1.0
1241710,40080,6,85,2009,22601 Voss Ave,California,Santa Clara,Cupertino,2012-04-18,Parts per billion,...,Parts per billion,1.871429,2.0,8,NaN,Parts per million,0.413043,0.5,6,NaN
45012,45012,17,31,8003,1703 STATE ST.,Illinois,Cook,Calumet City (PU RR name Calumet Park (sta.)),2000-07-09,Parts per billion,...,Parts per billion,8.525000,12.6,17,NaN,Parts per million,0.329167,0.5,21,NaN
1323232,121602,49,47,5632,"Accessed from Dragon Road/145-S, Uintah Co, UT",Utah,Uintah,Not in a city,2012-04-19,Parts per billion,...,Parts per billion,-0.329167,-0.3,2,0.0,Parts per million,0.595833,0.6,0,NaN


---

# Section 2

Begin to transform the data.

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Conclusion

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)
